# LeNet Lab
![LeNet Architecture](lenet.png)
Source: Yan LeCun

## Load Data

Load the MNIST data, which comes pre-loaded with TensorFlow.

You do not need to modify this section.

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="1"
os.environ["THEANO_FLAGS"]="mode=FAST_RUN,device=cuda1,floatX=float32"

In [2]:
'''
#Train a simple deep CNN on the CIFAR10 small images dataset.

It gets to 75% validation accuracy in 25 epochs, and 79% after 50 epochs.
(it's still underfitting at that point, though).
'''

from __future__ import print_function
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.optimizers import SGD
from keras.utils import np_utils
import os

batch_size = 1000
num_classes = 10
epochs = 100
data_augmentation = True
num_predictions = 20
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'keras_cifar10_trained_model.h5'

# The data, split between train and test sets:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# Convert class vectors to binary class matrices.
y_train = np_utils.to_categorical(y_train, num_classes)
y_test = np_utils.to_categorical(y_test, num_classes)

Using Theano backend.
ERROR (theano.gpuarray): Could not initialize pygpu, support disabled
Traceback (most recent call last):
  File "/usr/local/miniconda2/lib/python2.7/site-packages/theano/gpuarray/__init__.py", line 227, in <module>
    use(config.device)
  File "/usr/local/miniconda2/lib/python2.7/site-packages/theano/gpuarray/__init__.py", line 214, in use
    init_dev(device, preallocate=preallocate)
  File "/usr/local/miniconda2/lib/python2.7/site-packages/theano/gpuarray/__init__.py", line 99, in init_dev
    **args)
  File "pygpu/gpuarray.pyx", line 658, in pygpu.gpuarray.init
  File "pygpu/gpuarray.pyx", line 587, in pygpu.gpuarray.pygpu_init
GpuArrayException: cuDeviceGet: CUDA_ERROR_INVALID_DEVICE: invalid device ordinal


x_train shape: (50000, 3, 32, 32)
50000 train samples
10000 test samples


In [3]:
import numpy as np
def rgb2gray(rgb):
    r, g, b = rgb[:,0,:,:], rgb[:,1,:,:], rgb[:,2,:,:]
    gray = 0.2989 * r + 0.5870 * g + 0.1140 * b
    gray = np.expand_dims(gray, axis=3)
    print(gray.shape)
    return gray

In [4]:
x_train=rgb2gray(x_train)
x_test=rgb2gray(x_test)

(50000, 32, 32, 1)
(10000, 32, 32, 1)


The MNIST data that TensorFlow pre-loads comes as 28x28x1 images.

However, the LeNet architecture only accepts 32x32xC images, where C is the number of color channels.

In order to reformat the MNIST data into a shape that LeNet will accept, we pad the data with two rows of zeros on the top and bottom, and two columns of zeros on the left and right (28+2+2 = 32).

You do not need to modify this section.

## Preprocess Data

Shuffle the training data.

You do not need to modify this section.

In [5]:
from sklearn.utils import shuffle

X_train, y_train = shuffle(x_train, y_train)

In [6]:
print(X_train.shape)

(50000, 32, 32, 1)


## Setup TensorFlow
The `EPOCH` and `BATCH_SIZE` values affect the training speed and model accuracy.

You do not need to modify this section.

In [7]:
import tensorflow as tf

EPOCHS = 100
BATCH_SIZE = 1000

## TODO: Implement LeNet-5
Implement the [LeNet-5](http://yann.lecun.com/exdb/lenet/) neural network architecture.

This is the only cell you need to edit.
### Input
The LeNet architecture accepts a 32x32xC image as input, where C is the number of color channels. Since MNIST images are grayscale, C is 1 in this case.

### Architecture
**Layer 1: Convolutional.** The output shape should be 28x28x6.

**Activation.** Your choice of activation function.

**Pooling.** The output shape should be 14x14x6.

**Layer 2: Convolutional.** The output shape should be 10x10x16.

**Activation.** Your choice of activation function.

**Pooling.** The output shape should be 5x5x16.

**Flatten.** Flatten the output shape of the final pooling layer such that it's 1D instead of 3D. The easiest way to do is by using `tf.contrib.layers.flatten`, which is already imported for you.

**Layer 3: Fully Connected.** This should have 120 outputs.

**Activation.** Your choice of activation function.

**Layer 4: Fully Connected.** This should have 84 outputs.

**Activation.** Your choice of activation function.

**Layer 5: Fully Connected (Logits).** This should have 10 outputs.

### Output
Return the result of the 2nd fully connected layer.

In [8]:

def LeNet(x):    
    # Hyperparameters
    mu = 0
    sigma = 0.1
    layer_depth = {
        'layer_1' : 6,
        'layer_2' : 16,
        'layer_3' : 120,
        'layer_f1' : 84
    }

    
    # TODO: Layer 1: Convolutional. Input = 32x32x1. Output = 28x28x6.
    conv1_w = tf.Variable(tf.truncated_normal(shape = [5,5,1,6],mean = mu, stddev = sigma))
    conv1_b = tf.Variable(tf.zeros([6]))
    conv1 = tf.nn.conv2d(x,conv1_w, strides = [1,1,1,1], padding = 'VALID') + conv1_b 
    # TODO: Activation.
    conv1 = tf.nn.relu(conv1)

    # TODO: Pooling. Input = 28x28x6. Output = 14x14x6.
    pool_1 = tf.nn.max_pool(conv1,ksize = [1,2,2,1], strides = [1,2,2,1], padding = 'VALID')
    
    # TODO: Layer 2: Convolutional. Output = 10x10x16.
    conv2_w = tf.Variable(tf.truncated_normal(shape = [5,5,6,16], mean = mu, stddev = sigma))
    conv2_b = tf.Variable(tf.zeros([16]))
    conv2 = tf.nn.conv2d(pool_1, conv2_w, strides = [1,1,1,1], padding = 'VALID') + conv2_b
    # TODO: Activation.
    conv2 = tf.nn.relu(conv2)

    # TODO: Pooling. Input = 10x10x16. Output = 5x5x16.
    pool_2 = tf.nn.max_pool(conv2, ksize = [1,2,2,1], strides = [1,2,2,1], padding = 'VALID') 
    
    # TODO: Flatten. Input = 5x5x16. Output = 400.
    fc1 = tf.reshape(pool_2, [-1, 400])
    
    # TODO: Layer 3: Fully Connected. Input = 400. Output = 120.
    fc1_w = tf.Variable(tf.truncated_normal(shape = (400,120), mean = mu, stddev = sigma))
    fc1_b = tf.Variable(tf.zeros([120]))
    fc1 = tf.matmul(fc1,fc1_w) + fc1_b
    
    # TODO: Activation.
    fc1 = tf.nn.relu(fc1)

    # TODO: Layer 4: Fully Connected. Input = 120. Output = 84.
    fc2_w = tf.Variable(tf.truncated_normal(shape = (120,84), mean = mu, stddev = sigma))
    fc2_b = tf.Variable(tf.zeros([84]))
    fc2 = tf.matmul(fc1,fc2_w) + fc2_b
    # TODO: Activation.
    fc2 = tf.nn.relu(fc2)
    
    # TODO: Layer 5: Fully Connected. Input = 84. Output = 10.
    fc3_w = tf.Variable(tf.truncated_normal(shape = (84,10), mean = mu , stddev = sigma))
    fc3_b = tf.Variable(tf.zeros([10]))
    logits = tf.matmul(fc2, fc3_w) + fc3_b
    return logits

In [9]:
# # from tensorflow.contrib.layers import flatten

# def LeNet(x):    
#     # Hyperparameters
#     mu = 0
#     sigma = 0.1
#     layer_depth = {
#         'layer_1' : 6,
#         'layer_2' : 16,
#         'layer_3' : 120,
#         'layer_f1' : 84
#     }

    
#     # TODO: Layer 1: Convolutional. Input = 32x32x1. Output = 28x28x6.
#     conv1_w = tf.Variable(tf.truncated_normal(shape = [5,5,1,6],mean = mu, stddev = sigma))
#     conv1_b = tf.Variable(tf.zeros(6))
#     conv1 = tf.nn.conv2d(x,conv1_w, strides = [1,1,1,1], padding = 'VALID') + conv1_b 
#     # TODO: Activation.
#     conv1 = tf.nn.relu(conv1)

#     # TODO: Pooling. Input = 28x28x6. Output = 14x14x6.
#     pool_1 = tf.nn.max_pool(conv1,ksize = [1,2,2,1], strides = [1,2,2,1], padding = 'VALID')
    
#     # TODO: Layer 2: Convolutional. Output = 10x10x16.
#     conv2_w = tf.Variable(tf.truncated_normal(shape = [5,5,6,16], mean = mu, stddev = sigma))
#     conv2_b = tf.Variable(tf.zeros(16))
#     conv2 = tf.nn.conv2d(pool_1, conv2_w, strides = [1,1,1,1], padding = 'VALID') + conv2_b
#     # TODO: Activation.
#     conv2 = tf.nn.relu(conv2)

#     # TODO: Pooling. Input = 10x10x16. Output = 5x5x16.
#     pool_2 = tf.nn.max_pool(conv2, ksize = [1,2,2,1], strides = [1,2,2,1], padding = 'VALID')
    
#     #################
#     # TODO: Layer 3: Convolutional. Output = 120.
#     conv3_w = tf.Variable(tf.truncated_normal(shape = [5,5,16,120], mean = mu, stddev = sigma))
#     conv3_b = tf.Variable(tf.zeros(120))
#     conv3 = tf.nn.conv2d(pool_2, conv3_w, strides = [1,1,1,1], padding = 'VALID') + conv3_b

    
#     # TODO: Activation.
#     conv3 = tf.nn.relu(conv3)
    
#     ##############
#     # TODO: Flatten. Input = 5x5x16. Output = 400.
#     # fc1 = flatten(pool_2)
#     fc1 = tf.squeeze(conv3)
    
# #     # TODO: Layer 3: Fully Connected. Input = 400. Output = 120.
# #     fc1_w = tf.Variable(tf.truncated_normal(shape = (400,120), mean = mu, stddev = sigma))
# #     fc1_b = tf.Variable(tf.zeros(120))
# #     fc1 = tf.matmul(fc1,fc1_w) + fc1_b
    
# #     # TODO: Activation.
# #     fc1 = tf.nn.relu(fc1)

#     # TODO: Layer 4: Fully Connected. Input = 120. Output = 84.
#     fc2_w = tf.Variable(tf.truncated_normal(shape = (120,84), mean = mu, stddev = sigma))
#     fc2_b = tf.Variable(tf.zeros(84))
#     fc2 = tf.matmul(fc1,fc2_w) + fc2_b
#     # TODO: Activation.
#     fc2 = tf.nn.relu(fc2)
    
#     # TODO: Layer 5: Fully Connected. Input = 84. Output = 10.
#     fc3_w = tf.Variable(tf.truncated_normal(shape = (84,10), mean = mu , stddev = sigma))
#     fc3_b = tf.Variable(tf.zeros(10))
#     logits = tf.matmul(fc2, fc3_w) + fc3_b
#     return logits

## Features and Labels
Train LeNet to classify [MNIST](http://yann.lecun.com/exdb/mnist/) data.

`x` is a placeholder for a batch of input images.
`y` is a placeholder for a batch of output labels.

You do not need to modify this section.

In [10]:
y_train=y_train.astype(np.int)
y_test=y_test.astype(np.int)
# print(y_train[0])
# from sklearn.preprocessing import OneHotEncoder
# one_hot_encoder = OneHotEncoder(sparse= False)
# one_hot_encoder.fit(y_train)
# one_hot_y = one_hot_encoder.transform(y_train)
# print(one_hot_y.shape)
# one_hot_y = tf.convert_to_tensor(one_hot_y, dtype=tf.float32)
# one_hot_encoder.fit(y_test)
# one_hot_test = one_hot_encoder.transform(y_test)
# print(one_hot_test.shape)
# print(y_test.shape)
# #print(np.unique(y_test))
# one_hot_test = tf.convert_to_tensor(one_hot_test, dtype=tf.float32)
# x = tf.placeholder(tf.float32, (None, 32, 32, 1))
# y = tf.placeholder(tf.int32, (None))
# one_hot_y = tf.convert_to_tensor(one_hot_y, dtype=tf.float32)

In [17]:
x = tf.placeholder(tf.float32, (None, 32, 32, 1))
y = tf.placeholder(tf.float32, (None))
one_hot_y = y
# one_hot_y = tf.convert_to_tensor(y_train, dtype=tf.int32)
# one_hot_test = tf.convert_to_tensor(y_test, dtype=tf.float32)
# one_hot_y = y

## Training Pipeline
Create a training pipeline that uses the model to classify MNIST data.

You do not need to modify this section.

In [30]:
rate = 0.01

logits = LeNet(x)
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits = logits, labels = y)
loss_operation = tf.reduce_mean(cross_entropy)
optimizer = tf.train.GradientDescentOptimizer(learning_rate = rate)
training_operation = optimizer.minimize(loss_operation)

## Model Evaluation
Evaluate how well the loss and accuracy of the model for a given dataset.

You do not need to modify this section.

In [32]:
correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1))
accuracy_operation = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
saver = tf.train.Saver()

def evaluate(X_data, y_data):
    num_examples = len(X_data)
    total_accuracy = 0
    sess = tf.get_default_session()
    for offset in range(0, num_examples, BATCH_SIZE):
        batch_x, batch_y = X_data[offset:offset+BATCH_SIZE], y_data[offset:offset+BATCH_SIZE]
        accuracy = sess.run(accuracy_operation, feed_dict={x: batch_x, y: batch_y})
        total_accuracy += (accuracy * len(batch_x))
    return total_accuracy / num_examples

## Train the Model
Run the training data through the training pipeline to train the model.

Before each epoch, shuffle the training set.

After each epoch, measure the loss and accuracy of the validation set.

Save the model after training.

You do not need to modify this section.

In [33]:
# sess = tf.InteractiveSession()
# tf.initialize_all_variables().run()
# num_examples = len(X_train)
    
# print("Training...")
# print()
# for i in range(EPOCHS):
#     X_train, y_train = shuffle(X_train, y_train)
#     for offset in range(0, num_examples, BATCH_SIZE):
#         end = offset + BATCH_SIZE
#         batch_x, batch_y = X_train[offset:end], y_train[offset:end]
#         sess.run(training_operation, feed_dict={x: batch_x, y: batch_y})

#     validation_accuracy = evaluate(X_validation, y_validation)
#     print("EPOCH {} ...".format(i+1))
#     print("Validation Accuracy = {:.3f}".format(validation_accuracy))
#     print()

# saver.save(sess, './lenet')
# print("Model saved")

In [34]:
print(y_train.shape)

(50000, 10)


In [35]:
init = tf.initialize_all_variables()
with tf.Session() as sess:
    sess.run(init)
    num_examples = len(X_train)
    
    print("Training...")
    print()
    for i in range(EPOCHS):
        X_train, y_train = shuffle(X_train, y_train)
        for offset in range(0, num_examples, BATCH_SIZE):
            end = offset + BATCH_SIZE
            batch_x, batch_y = X_train[offset:end], y_train[offset:end]
            sess.run(training_operation, feed_dict={x: batch_x, y: batch_y})
            
        validation_accuracy = evaluate(x_test, y_test)
        print("EPOCH {} ...".format(i+1))
        print("Validation Accuracy = {:.3f}".format(validation_accuracy))
        print()
        
    saver.save(sess, './lenet')
    print("Model saved")

Training...

EPOCH 1 ...
Validation Accuracy = 0.100

EPOCH 2 ...
Validation Accuracy = 0.100

EPOCH 3 ...
Validation Accuracy = 0.100

EPOCH 4 ...
Validation Accuracy = 0.100

EPOCH 5 ...
Validation Accuracy = 0.100

EPOCH 6 ...
Validation Accuracy = 0.100

EPOCH 7 ...
Validation Accuracy = 0.100

EPOCH 8 ...
Validation Accuracy = 0.100

EPOCH 9 ...
Validation Accuracy = 0.100

EPOCH 10 ...
Validation Accuracy = 0.100

EPOCH 11 ...
Validation Accuracy = 0.100

EPOCH 12 ...
Validation Accuracy = 0.100

EPOCH 13 ...
Validation Accuracy = 0.100

EPOCH 14 ...
Validation Accuracy = 0.100

EPOCH 15 ...
Validation Accuracy = 0.100

EPOCH 16 ...
Validation Accuracy = 0.100

EPOCH 17 ...
Validation Accuracy = 0.100

EPOCH 18 ...
Validation Accuracy = 0.100

EPOCH 19 ...
Validation Accuracy = 0.100

EPOCH 20 ...
Validation Accuracy = 0.100

EPOCH 21 ...
Validation Accuracy = 0.100

EPOCH 22 ...
Validation Accuracy = 0.100

EPOCH 23 ...
Validation Accuracy = 0.100

EPOCH 24 ...
Validation Accura

## Evaluate the Model
Once you are completely satisfied with your model, evaluate the performance of the model on the test set.

Be sure to only do this once!

If you were to measure the performance of your trained model on the test set, then improve your model, and then measure the performance of your model on the test set again, that would invalidate your test results. You wouldn't get a true measure of how well your model would perform against real data.

You do not need to modify this section.

In [ ]:
with tf.Session() as sess:
    saver.restore(sess, tf.train.latest_checkpoint('.'))

    test_accuracy = evaluate(X_test, y_test)
    print("Test Accuracy = {:.3f}".format(test_accuracy))